In [ ]:
import gzip
import hashlib
import requests
import csv
from qnt.output_avro import convert_avro_record_to_output, convert_output_to_avro_record
from qnt.data import sort_and_crop_output

In [ ]:
# Feed API avro schema
feed_api = "http://quantnet-ai.ru:9090/statan/feed/avro/"

# Request authorization header (PUT YOUR PERSONAL TOKEN HERE)
feed_api_header = "YOUR ACCESS TOKEN"

# Blockchain public API which should return all information about verification transactions and dates
blockchain_api = "http://192.168.122.241:5300/getAllData"

In [ ]:
# Fetching data from Blockchain API
blockchain_api_data = requests.get(blockchain_api).json()["result"]

# Creating empty arrays for storing blockchain data
# Blockchain transactions hashes
txs = []

# Sha256 root hashes generated from daily data
roots = []

# CSV texts of submisions (submission_id, daily_output_avro_hash)
csv_texts = []

# Dates of each csv text, fetched from the Statan API
dates = []

# Sorting data into arrays by type
for data in blockchain_api_data:
    txs.append(data["tx"])
    roots.append(data["root"])
    csv_texts.append(data["data"])
    dates.append(data["date"])

In [34]:
# Hashing the daily data with a sha256 algorithm, and compare it with provided data from API 
def compare_daily_data_hash_with_generated_hash(root, data):
    hash = hashlib.sha256(data.encode('utf-8')).hexdigest()
    return bool(True) if hash == root else bool(False)

# verifieng all dates and hashes
# as an argument need to pass root hashes and csv texts
# the lengths of arrays should be equal
def compare_root_hashes_with_daily_data_hashes(roots, csv_texts):
    if (len(roots) == len(csv_texts)):
        i = 0
        while i < len(csv_texts):
            is_valid = compare_daily_data_hash_with_generated_hash(roots[i], csv_texts[i])
            result = "the same" if is_valid else "different"
            print("The hash of csv data and root from " + dates[i] + " is " + result)
            i += 1
    else:
        print("The lengths should be equal!")

In [ ]:
# Fetching avro/binary data by submission_id from Feed API
def get_submision_output(submission_id):    
    output = requests.get(feed_api + submission_id, headers={"Authorization" : feed_api_header})
    output = convert_avro_record_to_output(output.content)
    return output

# Generate the sha256 hash by submission_id for certain date
def generate_output_hash(submission_id, date):
    xarr = get_submision_output(submission_id)
    s = xarr.loc[date:date]
    s = sort_and_crop_output(s)
    avs = convert_output_to_avro_record(s)
    hash = hashlib.sha256(avs).hexdigest()
    return hash

# Generate the sha256 hash from output and verify with daily output hash
def verify_outputs_per_date(date):
    data = csv_texts[dates.index(date)]
    csv_data = csv.DictReader(data.splitlines())
    for row in csv_data:
        submission_id = row["submission_id"]
        avro_hash = row["daily_output_avro_hash"]
        generated_hash = generate_output_hash(submission_id, date)
        result = "valid" if generated_hash == avro_hash else "invalid"
        print("ID " + submission_id + " at " + date + " is " + result)

# Verify all outputs from all dates
def verify_outputs_all_dates(dates):
    for date in dates:
        verify_outputs_per_date(date)

In [ ]:
# Verification of blockchain based roots and daily data
compare_root_hashes_with_daily_data_hashes(roots, csv_texts)

In [ ]:
# Verification of the generated sha256 hash of the daily data and provided sha256 hashes
verify_outputs_all_dates(dates)